<a href="https://colab.research.google.com/github/profteachkids/CHE2064/blob/master/Demo_AdiabaticFlash.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!wget https://raw.githubusercontent.com/profteachkids/CHE2064/master/vsc.py
!wget https://raw.githubusercontent.com/profteachkids/CHE2064/master/trust_ncg.py
!wget https://raw.githubusercontent.com/profteachkids/CHE2064/master/che_tools.py
!pip install DotMap

--2020-09-08 12:20:11--  https://raw.githubusercontent.com/profteachkids/CHE2064/master/vsc.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3247 (3.2K) [text/plain]
Saving to: ‘vsc.py’

vsc.py              100%[===================>]   3.17K  --.-KB/s    in 0s      

2020-09-08 12:20:11 (49.1 MB/s) - ‘vsc.py’ saved [3247/3247]

--2020-09-08 12:20:11--  https://raw.githubusercontent.com/profteachkids/CHE2064/master/trust_ncg.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2871 (2.8K) [text/plain]
Saving to: ‘trust_ncg.py’

trust_ncg.py 

In [22]:
from dotmap import DotMap
import jax.numpy as jnp
import jax
import vsc
import trust_ncg as tncg
import che_tools as che

In [23]:
p = che.Props(['Ethanol', 'Isopropanol', 'Water'])

In [24]:
def model(c):
    c=DotMap(c)
    V = c.Vy * c.Vtot
    L = c.Lx * c.Ltot
    F = c.Fz * c.Ftot
    mass_balance = (F - V - L)
    mass_balance = (mass_balance/jnp.sum(F))**2

    FH = p.Hmix(0., F, c.FT)
    VH = p.Hmix(V, 0., c.flashT)
    LH = p.Hmix(0., L, c.flashT)
    energy_balance = (FH - VH - LH)
    energy_balance = (energy_balance/jnp.sum(FH))**2

    fugacity = (c.Lx*p.Pvap(c.flashT) - c.Vy*c.flashP)
    fugacity = (fugacity/jnp.sum(c.Vy*c.flashP))**2

    return jnp.sum(mass_balance + energy_balance + fugacity)

In [25]:
s=DotMap()
s.Ftot=10
s.Fz = jnp.array([1/3, 1/3, 1/3])
s.FT = 380
s.flashP= 101325

v=DotMap()
v.Vy = vsc.Comp([0.4, 0.35, 0.25])
v.Lx = vsc.Comp([0.25, 0.35, 0.4])
v.flashT = vsc.Range(350, 300, 380)
v.Vtot = s.Ftot/2
v.Ltot = s.Ftot/2

In [26]:
c=vsc.VSC(v,s)
model(c.xtoc(c.x))

["<class 'int'>, 380\n", "<class 'int'>, 10\n", "<class 'jax.interpreters.xla.DeviceArray'>, [0.33333333 0.33333333 0.33333333]\n", "<class 'float'>, 5.0\n", "<class 'numpy.float64'>, -0.47000362924573547\n", "<class 'numpy.float64'>, -0.13353139262452274\n", "<class 'float'>, 5.0\n", "<class 'numpy.float64'>, 0.4700036292457356\n", "<class 'numpy.float64'>, 0.33647223662121273\n", "<class 'int'>, 101325\n", "<class 'float'>, 0.625\n"]
["<class 'float'>, nan\n", "<class 'float'>, nan\n", "<class 'jax.interpreters.xla.DeviceArray'>, [nan nan nan]\n", "<class 'float'>, 5.0\n", "<class 'numpy.float64'>, -0.47000362924573547\n", "<class 'numpy.float64'>, -0.13353139262452274\n", "<class 'float'>, 5.0\n", "<class 'numpy.float64'>, 0.4700036292457356\n", "<class 'numpy.float64'>, 0.33647223662121273\n", "<class 'float'>, nan\n", "<class 'float'>, 0.625\n"]


DeviceArray(0.0721327, dtype=float64)

In [27]:
modelf = jax.jit(c.transform(model))
x_min, f = tncg.minimize(modelf,c.x, verbosity=1)

0:2, f: 0.0721327005499455
1:1, f: 0.0721327005499455
2:0, f: 0.0721327005499455
3:1, f: 0.0495429252974428
4:1, f: 0.03640191500521994
5:2, f: 0.026654456423456013
6:4, f: 0.014158431599687736
7:4, f: 0.008162012342045732
8:7, f: 0.0032456943639230665
9:7, f: 0.0001357743225833637
10:7, f: 1.588911353409773e-07
Final results:
f: 1.158868877130111e-13
x: [ 9.19945904 -0.05913598 -0.04393861  0.80054178  0.75797847  0.61460816
  0.73290355]


In [28]:
v_sol = c.xtoc(x_min)
df=vsc.todf(v_sol)
df

,,"(vector3, 1)","(vector3, 2)","(vector3, 3)"
FT,380,,,
Ftot,10,,,
Fz,,0.333333,0.333333,0.333333
Ltot,9.19946,,,
Lx,,0.325073,0.330051,0.344876
Vtot,0.800542,,,
Vy,,0.428257,0.371056,0.200687
flashP,101325,,,
flashT,358.632,,,
